In [1]:
import keras_tuner as kt
import numpy as np

from tensorflow.keras.callbacks import Callback

import IPython

In [2]:
from model import tuner_model, DataGenerator

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080, compute capability 8.6


In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
np.random.seed(42)

In [5]:
from simple_loader import dataloader
x_dataset, y_dataset = dataloader('./pickle', 'finance_scaled', ['x_dataset_series', 'y_dataset_series'])
x_dataset.shape, y_dataset.shape

((5996058, 32, 22), (5996058, 32))

In [6]:
def shuffle_arrays(*arrays):
    rng_state = np.random.get_state()
    for array in arrays:
        np.random.set_state(rng_state)
        np.random.shuffle(array)

In [7]:
# shuffle_arrays(x_dataset, y_dataset)

In [6]:
validation_split = 0.2
val_len = int(validation_split * x_dataset.shape[0])

val_gen = DataGenerator(x_dataset[-val_len:], y_dataset[-val_len:], 2048)
train_gen = DataGenerator(x_dataset[:-val_len], y_dataset[:-val_len], 2048, shuffle=True)

In [7]:
class ClearTrainingOutput(Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [9]:
tuner = kt.Hyperband(
    tuner_model,
    objective = 'val_loss', 
    max_epochs = 10,
    factor = 3,
    directory = './tuner',
    project_name = 'academy_tuner_scaled_32-series'
)

In [10]:
tuner.search(
    train_gen,
    epochs = 10,
    validation_data = val_gen,
    shuffle = True,
    callbacks = [ClearTrainingOutput()]
)

Trial 30 Complete [01h 34m 44s]
val_loss: 0.02652868814766407

Best val_loss So Far: 0.006245271768420935
Total elapsed time: 09h 11m 51s
INFO:tensorflow:Oracle triggered exit


In [11]:
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "finance"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
finance_data (InputLayer)       [(None, 32, 22)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 32, 128)      8576        finance_data[0][0]               
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 128)      512         conv1d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 32, 128)      0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [12]:
best_hps = tuner.get_best_hyperparameters(1)[0]

print(f"""
The hyperparameter search is complete.
conv_blocks {best_hps.get('conv_blocks')}
filters {best_hps.get('filters')}
kernel_size {best_hps.get('kernel_size')}
dense_blocks {best_hps.get('dense_blocks')}
units {best_hps.get('units')}
l2_val {best_hps.get('l2_val')}
dropout_rate {best_hps.get('dropout_rate')}
learning rate {best_hps.get('learning_rate')}.
""")



The hyperparameter search is complete.
conv_blocks 4
filters 128
kernel_size 3
dense_blocks 1
units 256
l2_val 1e-6
dropout_rate 0.1
learning rate 0.0005.



In [ ]:
# The hyperparameter search is complete. 32-series
# conv_blocks 4
# filters 128
# kernel_size 3
# dense_blocks 1
# units 256
# l2_val 1e-6
# dropout_rate 0.1
# learning rate 0.0005.

In [14]:
# The hyperparameter search is complete. 4-series
# conv_blocks 8
# filters 32
# kernel_size 11
# dense_blocks 1
# units 32
# l2_val None
# dropout_rate 0.3
# learning rate 0.0001.


In [15]:
import numpy as np
def shuffle_arrays(*arrays):
    rng_state = np.random.get_state()
    for array in arrays:
        np.random.set_state(rng_state)
        np.random.shuffle(array)

In [16]:
N = 10000
a = np.random.randn(N, N)
b = a.copy()

shuffle_arrays(a, b)

(a == b).all()

True